<a href="https://colab.research.google.com/github/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/blob/main/Sentiment_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd

##Szószedet

In [33]:
!rm *zip*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip
df=pd.read_csv("TEST.csv")

--2021-10-06 17:36:29--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-06 17:36:29--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [34]:
df.head()
alltext1=df.text
print (len(alltext1))

2640


In [35]:
df2=pd.read_csv("TRAIN.csv")
alltext2=df2.text
print (len(alltext2))

12000


In [31]:
alltext1=list(alltext1)
alltext2=list(alltext2)
alltext=alltext1+alltext2
print(f" {len(alltext)}")


 14640


In [36]:
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()
print(spell.correction("@frustrating"))

frustrating


In [37]:
import re

def remove_URL(text):
    """Remove URLs from a text string"""
    return re.sub(r"http\S+", "", text)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def is_number(inp:str):
    a=re.search("[\d]",inp)
    if a==None: 
        return(False)
    return(True)

def strip_all_entities(text):
    import re,string
    entity_prefixes = ['“','@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def query_sentence(index):
    print(f"orig: {alltext[index]}")
    spell=SpellChecker()
    inp=alltext[index]
    inp=remove_URL(inp)
    inp=remove_emoji(inp)
    inp=strip_all_entities(inp)
    
    txt=inp.replace(","," ").replace("."," ").replace("!"," ").replace("#"," ").replace("???"," ").replace("??"," ").replace("?"," ").replace("  "," ").replace("  "," ").replace('"',"").replace("“","")

    #print(txt)
    normal=""
    for i in txt.split(" "):
        w1=i.lower().strip()
        if w1 !="":
            w2=spell.correction(w1).strip()
        else:
            w2=""
        if w2=="i":
            pass
            #print(f"chek: {w1}, {w2}")
        if is_number(w2):
            w2=""    
        normal += w2+" "

    print(f"  conv: {normal}")
    return(normal)

##Mentés

In [ ]:
t=[]
for i in range(len(alltext)):
    print(f"{i}", end=" ")
    t.append(query_sentence(i))


In [49]:
s=set()
for sent in t:
    wlist=sent.strip().split()
    for w in wlist:
        s.add(w)
print (len(s))
ordered=list(s)
ordered.sort()

9098


In [ ]:
print(ordered)

In [63]:
sentence=pd.DataFrame(t)
sentence.columns=["Sentence"]
sentence.to_csv("Sentences.csv")

In [55]:
wordlist=pd.DataFrame(ordered)
wordlist.columns=["Words"]
wordlist.to_csv("words.csv")

###---------------------------------


###Adatbetöltés SZAVAK BETÖLTÉSE

In [64]:
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv"

In [65]:
!wget $words_url

--2021-10-06 19:00:18--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv [following]
--2021-10-06 19:00:18--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114745 (112K) [text/plain]
Saving to: ‘words.csv’

words.csv           100%[=================

In [66]:
words=pd.read_csv("words.csv",index_col=0)
words.head()
words_list=list(words["Words"])

In [67]:
word_dict={v:i+1 for i,v in enumerate(words_list)}

##Tanulás

In [ ]:
sentence.head()

In [70]:
df=pd.read_csv("TRAIN.csv")

In [ ]:
df.head()

In [ ]:
table=list(df["sentence_code"]) # typo
lens=max([len(x) for x in table])
sentiment=list(df["airline_sentiment"])


In [ ]:
ytrain=[]
for sent in sentiment:
    if sent=="negative":
        o=[1,0,0]
    if sent=="neutral":
        o=[0,1,0]
    if sent=="positive":
        o=[0,0,1]
    ytrain.append(o)
    

In [ ]:
x0=[0 for _ in range(lens) ]
xtrain=[]
for sent in table:
    o1=list(x0[0:lens-len(sent)]+list(sent))
    xtrain.append(o1)

In [ ]:
print(lens)

In [ ]:


lstm_size=lens
max_input_length=lens
embedding_size=100
n_words=len(table)

In [ ]:

# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy
from tensorflow.keras import backend as K
# Gondolj bele, hogy a hálóban az első réteg egy "beágazás" kell majd legyen
# Ne feledd behozni a funkcionális vagy szekvenciális API-nak megfelelő "fő" osztályt
# Adott esetben az optimalizálót
# Nem külömben a "bakcendet", hogy jó gyakorlat szerint reseteld a gráfot
#tf.reset_default_graph()
K.clear_session
x= Input(shape=(max_input_length,))
embedded_x=Embedding(n_words+1,embedding_size, input_length=max_input_length-1, mask_zero=True)(x)
lstm_output=LSTM(lstm_size,return_sequences=True)(embedded_x)
lstm_output=LSTM(lstm_size,return_sequences=False)(lstm_output)
predictions= Dense(3, activation="softmax")(lstm_output)
model=Model(inputs=x, outputs=predictions)



In [ ]:
model.summary()

In [ ]:
x

In [ ]:
import numpy as np

In [ ]:
x_train=xtrain[0:-2000]
y_train=ytrain[0:-2000]
x_test=xtrain[-2000:]
y_test=ytrain[-2000:]

In [ ]:
train_x = np.asarray(xtrain)
train_y = np.asarray(ytrain)

In [ ]:
# Loss 

loss = categorical_crossentropy # One-hot enkódolt kimenetünk van. Mit is használunk?

# Optimizer
optimizer = Adam() #Ízlés szerint...
 
# Compilation
#############

model.compile(optimizer=optimizer,loss=loss)

In [ ]:
# Illesszük az adatra a modellt. Használjunk 10% validációt
history=model.fit(x=xtrain,y=ytrain,validation_data=( x_test,y_test), epochs=25, batch_size=100)
# - nyelvmodellnél ez nem olyan lényeges
# Használhatjuk a Keras beépített validációs splitjét.
# Adjunk meg reális batch méretet!

In [ ]:
pred=model.predict(x_train)

In [ ]:
for i in range(len(pred)):
    print(f"{i}, {pred[i]}, {y_train[i]}")

